In [43]:
from typing import TypedDict, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.types import interrupt,Command
from typing import List
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

In [44]:
class State(TypedDict):
    jira_key: str
    feature_file:str
    status: str
    approved: bool

In [45]:
def get_jira_key(state: State) -> State:
    return {"jira_key": "QCSG-567"}

def get_feature_file(state: State) -> State:
    
    state["feature_file"] = "This is a feature file"
    if not state["approved"]:
        interrupt("awaiting approval")
    return {"feature_file": "This is a feature file"}

def execute_task(state:State) -> State:
    return {"status": "completed"}

In [46]:
workflow = StateGraph(State)

workflow.add_node("jira_node",get_jira_key)
workflow.add_node("feature_node", get_feature_file)
workflow.add_node("execute_task",execute_task)

workflow.add_edge("jira_node","feature_node" )
workflow.add_edge("feature_node","execute_task" )
workflow.add_edge("execute_task",END)

workflow.set_entry_point("jira_node")

checkpointer = SqliteSaver(sqlite3.connect("sample_checkpointer.db", check_same_thread=False))
graph = workflow.compile(checkpointer=checkpointer)


In [47]:


initial_state= {
    "jira_key": "",
    "feature_file":"",
    "status": "",
    "approved": False
}

graph = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "thread-1"}}
result = graph.invoke(initial_state,config=config)
result


{'jira_key': 'QCSG-567',
 'feature_file': '',
 'status': '',
 'approved': False,
 '__interrupt__': [Interrupt(value='awaiting approval', resumable=True, ns=['feature_node:19273f9c-af4e-65a0-9357-53f0450c485b'])]}

In [48]:
result=graph.invoke(Command(resume= {"approved":True}),config=config)
result

{'jira_key': 'QCSG-567',
 'feature_file': 'This is a feature file',
 'status': 'completed',
 'approved': False}

In [49]:
checkpointer.conn.close()